In [26]:
template = {
    'user_id': int,
    'name': {
        'first': str,
        'last': str
    },
    'bio': {
        'dob': {
            'year': int,
            'month': int,
            'day': int
        },
        'birthplace': {
            'country': str,
            'city': str
        }
    }
}

john = {
    'user_id': 100,
    'name': {
        'first': 'John',
        'last': 'Cleese'
    },
    'bio': {
        'dob': {
            'year': 1939,
            'month': 11,
            'day': 27
        },
        'birthplace': {
            'country': 'United Kingdom',
            'city': 'Weston-super-Mare'
        }
    }
}


def find_mismatched_key(data, template):
    keys_data = data.keys()
    keys_template = template.keys()
    mismatched_keys = list((keys_data - keys_template) or (keys_template - keys_data))
    try:
        return 'mismatched keys', mismatched_keys[0]
    except IndexError:
        return '', None


def is_same_type(data, original_type):
    try:
        return isinstance(data, original_type)
    except TypeError:
        return isinstance(data, dict) and isinstance(original_type, dict)


def find_mistyped_key(data, template):
    for key in template:
        if not is_same_type(data[key], template[key]):
            return 'bad types', key
    return '', None


def render_error_trace(key_default, key_error):
    if not key_error:
        return ''
    preceed_key = key_default if key_default else ''
    is_connected = '.' if preceed_key else ''
    return f'{preceed_key}{is_connected}{key_error}'


def validate(data, template):
    error_type = ''

    def inner(data, template, key_default):
        nonlocal error_type
        error_trace = ''
        key_error = None

        error_type, key_error = find_mismatched_key(data, template)
        if not key_error:
            error_type, key_error = find_mistyped_key(data, template)
        error_trace = render_error_trace(key_default, key_error)

        if not error_trace:
            for key in data:
                if isinstance(data[key], dict):
                    inner_error_trace = inner(data[key], template[key], key)
                    if inner_error_trace:
                        error_trace = render_error_trace(key_default, inner_error_trace)
                        break

        return error_trace

    error_trace = inner(data, template, None)
    return (True, None) if not error_type \
                        else (False, f'{error_type}: {error_trace}')


print(validate(john, template))


(True, None)
